# Impact of Neighborhood Size on NCA Models

This notebook performs a comprehensive scientific analysis to determine whether it makes sense to use a `neighborhood_size` greater than 3, and what are the differences between models with different neighborhood sizes.

## Analysis Objectives:
1. **Performance Evaluation**: Comparison of biological metrics across different neighborhood sizes
2. **Trend Analysis**: Identification of patterns and improvements/degradations
3. **Computational Complexity**: Analysis of computational cost vs. benefits
4. **Interactive Visualizations**: Plotly charts for in-depth exploration
5. **Rule specialization analysis**: Analyze how Mixture NCA and Stochastic Mixture NCA rules specialize (or lose specialization) with different neighborhood sizes.
6. **Statistical significance Tests**: Perform formal statistical tests to determine if differences between neighborhood sizes are statistically significant


### Imports

In [85]:
import sys
import os
from pathlib import Path

from experiments.analyze_neighborhood_sizes import NeighborhoodSizeAnalyzer

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [86]:
# Add parent directory to path
# Get the directory where this notebook is located
notebook_dir = Path().absolute()
# Get the project root (parent of notebooks directory)
project_root = notebook_dir.parent
# Add to path
sys.path.insert(0, str(project_root))
sys.path.insert(0, str(project_root / 'experiments'))

print("Imports completed")
print(f"Project root: {project_root}")

Imports completed
Project root: /Users/luigidaddario/Documents/GitHub/MNCA


## Configuration

Define the parameters for the analysis:


In [ ]:
# Configuration
# Use absolute paths based on project root
# If running from notebooks/, go up one level to project root
if 'notebooks' in str(notebook_dir):
    project_root = notebook_dir.parent
else:
    project_root = notebook_dir

RESULTS_DIR = str(project_root / "experiments" / "results_extended")
HISTORIES_PATH = str(project_root / "histories.npy")
DEVICE = "auto"  # "auto", "cuda", "mps", or "cpu"
N_EVALUATIONS = 1000  # 1000 simulazioni per ogni modello
NEIGHBORHOOD_SIZES = [1, 2, 3, 4, 5, 6, 7]  # Test da 1 a 7 neighborhoods
STEP_LENGTHS = [35, 100, 500]  # Test con 35, 100, 500 step
FORCE_RECOMPUTE = False  # If True, re-evaluate even if CSV files exist
GENERATE_VIDEOS = True  # Genera video della generazione

print(f"Notebook directory: {notebook_dir}")
print(f"Project root: {project_root}")
print(f"Results directory: {RESULTS_DIR}")
print(f"Histories path: {HISTORIES_PATH}")
print(f"Device: {DEVICE}")
print(f"Neighborhood sizes: {NEIGHBORHOOD_SIZES}")
print(f"Number of evaluations: {N_EVALUATIONS}")
print(f"Step lengths: {STEP_LENGTHS}")
print(f"Generate videos: {GENERATE_VIDEOS}")
print(f"Paths exist: RESULTS_DIR={os.path.exists(RESULTS_DIR)}, HISTORIES={os.path.exists(HISTORIES_PATH)}")

Notebook directory: /Users/luigidaddario/Documents/GitHub/MNCA/notebooks
Project root: /Users/luigidaddario/Documents/GitHub/MNCA
Results directory: /Users/luigidaddario/Documents/GitHub/MNCA/experiments/results_extended
Histories path: /Users/luigidaddario/Documents/GitHub/MNCA/histories.npy
Device: auto
Neighborhood sizes: [3, 4, 5, 6, 7]
Number of evaluations: 30
Paths exist: RESULTS_DIR=True, HISTORIES=True


## Analyzer Initialization

Create the analyzer instance and load/evaluate the models:


In [ ]:
analyzer = NeighborhoodSizeAnalyzer(
    results_dir=RESULTS_DIR,
    histories_path=HISTORIES_PATH,
    device=DEVICE,
    n_evaluations=N_EVALUATIONS,
    step_lengths=STEP_LENGTHS
)

analyzer.load_or_evaluate_models(
    neighborhood_sizes=NEIGHBORHOOD_SIZES,
    force_recompute=FORCE_RECOMPUTE
)

print("\n Models loaded/evaluated successfully")

Loading histories from /Users/luigidaddario/Documents/GitHub/MNCA/histories.npy...
Loaded 200 simulations

Loading/Evaluating Models

Loading existing metrics for nb_size=3...
Loading existing metrics for nb_size=4...
Loading existing metrics for nb_size=5...
Loading existing metrics for nb_size=6...
Loading existing metrics for nb_size=7...

Saved aggregated metrics to /Users/luigidaddario/Documents/GitHub/MNCA/experiments/results_extended/tissue_simulation_extended/all_neighborhood_sizes_metrics.csv

 Models loaded/evaluated successfully


## Video Generation

Generate videos showing the evolution of NCA models for different neighborhood sizes and step lengths:

In [ ]:
if GENERATE_VIDEOS:
    from experiments.tissue_simulation_extended import _generate_videos_for_models
    from mix_NCA.utils_simulations import grid_to_channels_batch
    from mix_NCA.ExtendedNCA import ExtendedNCA
    from mix_NCA.ExtendedMixtureNCA import ExtendedMixtureNCA
    from mix_NCA.ExtendedMixtureNCANoise import ExtendedMixtureNCANoise
    from mix_NCA.TissueModel import ComplexCellType
    import torch
    
    def make_update_net_fn(device):
        def update_net_wrapper(n_channels, hidden_dims=128, n_channels_out=None, device_arg=None):
            return classification_update_net(n_channels, hidden_dims, n_channels_out, device=device)
        return update_net_wrapper
    
    update_net_fn = make_update_net_fn(analyzer.device)
    
    print("Generating videos for all models...")
    for nb_size in NEIGHBORHOOD_SIZES:
        exp_dir = Path(RESULTS_DIR) / "tissue_simulation_extended" / f"NB_{nb_size}"
        
        if not exp_dir.exists():
            print(f"Skipping NB={nb_size}: directory not found")
            continue
        
        # Load models
        models = {}
        
        # Standard NCA
        std_path = exp_dir / 'standard_nca.pt'
        if std_path.exists():
            nca = ExtendedNCA(
                update_net=classification_update_net(6 * 3, n_channels_out=6, device=analyzer.device),
                hidden_dim=128,
                maintain_seed=False,
                use_alive_mask=False,
                state_dim=6,
                residual=False,
                neighborhood_size=nb_size,
                device=analyzer.device
            )
            nca.load_state_dict(torch.load(std_path, map_location=analyzer.device, weights_only=True))
            nca.eval()
            models['standard_nca'] = nca.to(analyzer.device)
        
        # Mixture NCA
        mix_path = exp_dir / 'mixture_nca.pt'
        if mix_path.exists():
            mix_nca = ExtendedMixtureNCA(
                update_nets=update_net_fn,
                hidden_dim=128,
                maintain_seed=False,
                use_alive_mask=False,
                state_dim=6,
                num_rules=5,
                residual=False,
                temperature=3,
                neighborhood_size=nb_size,
                device=analyzer.device
            )
            mix_nca.load_state_dict(torch.load(mix_path, map_location=analyzer.device, weights_only=True))
            mix_nca.eval()
            models['mixture_nca'] = mix_nca.to(analyzer.device)
        
        # Stochastic Mixture NCA
        stoch_path = exp_dir / 'stochastic_mix_nca.pt'
        if stoch_path.exists():
            stochastic_mix_nca = ExtendedMixtureNCANoise(
                update_nets=update_net_fn,
                hidden_dim=128,
                maintain_seed=False,
                use_alive_mask=False,
                state_dim=6,
                num_rules=5,
                residual=False,
                temperature=3,
                neighborhood_size=nb_size,
                device=analyzer.device
            )
            stochastic_mix_nca.load_state_dict(torch.load(stoch_path, map_location=analyzer.device, weights_only=True))
            stochastic_mix_nca.eval()
            models['stochastic_nca'] = stochastic_mix_nca.to(analyzer.device)
        
        # NCA with Noise
        noise_path = exp_dir / 'nca_with_noise.pt'
        if noise_path.exists():
            nca_with_noise = ExtendedNCA(
                update_net=classification_update_net(6 * 3, n_channels_out=6 * 2, device=analyzer.device),
                hidden_dim=128,
                maintain_seed=False,
                use_alive_mask=False,
                state_dim=6,
                residual=False,
                distribution="normal",
                neighborhood_size=nb_size,
                device=analyzer.device
            )
            nca_with_noise.random_updates = True
            nca_with_noise.load_state_dict(torch.load(noise_path, map_location=analyzer.device, weights_only=True))
            nca_with_noise.eval()
            models['nca_with_noise'] = nca_with_noise.to(analyzer.device)
        
        # Generate videos for each step length
        for n_steps in STEP_LENGTHS:
            print(f"\n  Generating videos for NB={nb_size}, steps={n_steps}...")
            _generate_videos_for_models(
                models=models,
                histories=analyzer.histories,
                n_steps=n_steps,
                nb_size=nb_size,
                output_dir=exp_dir,
                device=analyzer.device,
                n_samples=3
            )
    
    print("\n Video generation completed!")
else:
    print("Video generation is disabled (GENERATE_VIDEOS=False)")

## Data Exploration

Examine the metrics data:


In [91]:
df = analyzer.parse_metrics()

print("Dataset shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nFirst data:")
df.head(50)

Dataset shape: (20, 14)

Columns: ['Model Type', 'KL Divergence', 'KL Divergence SD', 'Chi-Square', 'Chi-Square SD', 'Categorical MMD', 'Categorical MMD SD', 'Tumor Size Diff', 'Tumor Size Diff SD', 'Border Size Diff', 'Border Size Diff SD', 'Spatial Variance Diff', 'Spatial Variance Diff SD', 'Neighborhood Size']

First data:


,Model Type,KL Divergence,KL Divergence SD,Chi-Square,Chi-Square SD,Categorical MMD,Categorical MMD SD,Tumor Size Diff,Tumor Size Diff SD,Border Size Diff,Border Size Diff SD,Spatial Variance Diff,Spatial Variance Diff SD,Neighborhood Size
0,Standard NCA,7.580,±0.000,0.984,±0.000,0.959,±0.000,1.119,±0.000,0.457,±0.000,1.046,±0.000,3
1,Mixture NCA,0.054,±0.001,0.053,±0.001,0.055,±0.003,0.441,±0.022,0.274,±0.012,0.656,±0.029,3
2,Stochastic Mixture NCA,0.135,±0.003,0.101,±0.001,0.017,±0.000,0.157,±0.012,0.061,±0.006,0.127,±0.010,3
3,NCA with Noise,0.223,±0.004,0.224,±0.003,0.202,±0.001,0.805,±0.001,0.113,±0.006,0.804,±0.012,3
4,Standard NCA,1.690,±0.000,0.757,±0.000,0.656,±0.000,0.390,±0.000,0.457,±0.000,1.046,±0.000,4
5,Mixture NCA,0.053,±0.003,0.049,±0.003,0.164,±0.005,0.450,±0.026,0.331,±0.007,0.735,±0.014,4
6,Stochastic Mixture NCA,0.033,±0.003,0.031,±0.002,0.143,±0.005,0.076,±0.018,0.228,±0.014,0.662,±0.022,4
7,NCA with Noise,0.090,±0.002,0.081,±0.002,0.172,±0.001,0.667,±0.002,0.323,±0.006,0.911,±0.010,4
8,Standard NCA,1.134,±0.000,0.515,±0.000,0.807,±0.000,0.217,±0.000,0.457,±0.000,1.046,±0.000,5
9,Mixture NCA,0.010,±0.001,0.010,±0.001,0.033,±0.001,0.212,±0.008,0.117,±0.009,0.338,±0.015,5


Descriptive statistics by model

In [92]:
for model_type in df['Model Type'].unique():
    print(f"\n{model_type}:")
    model_data = df[df['Model Type'] == model_type]
 
    numeric_cols = model_data.select_dtypes(include=[np.number]).columns.tolist()
    display_cols = [col for col in numeric_cols if col != 'Neighborhood Size']
    summary = model_data.set_index('Neighborhood Size')[display_cols]
    print(summary)


Standard NCA:
                   KL Divergence  Chi-Square  Categorical MMD  \
Neighborhood Size                                               
3                          7.580       0.984            0.959   
4                          1.690       0.757            0.656   
5                          1.134       0.515            0.807   
6                          0.744       0.423            0.611   
7                          0.218       0.200            0.675   

                   Tumor Size Diff  Border Size Diff  Spatial Variance Diff  
Neighborhood Size                                                            
3                            1.119             0.457                  1.046  
4                            0.390             0.457                  1.046  
5                            0.217             0.457                  1.046  
6                            0.387             0.457                  1.046  
7                            0.246             0.457         

## Trend Analysis

Analyze performance trends as neighborhood size varies:


In [94]:
trend_df = analyzer.performance_trend_analysis()

print("Trend Analysis:")

trend_df


Performance Trend Analysis

Standard NCA - KL Divergence:
  Correlation (Pearson): r=-0.8218, p=0.0879
  Correlation (Spearman): r=-1.0000, p=0.0000
  Linear trend: slope=-1.567000, p=0.0879
  Best NB: 7, Worst NB: 3
  Improvement NB3→NB7: 97.12%
  Fold Change (NB7/NB3): 0.029x
  Absolute Difference (NB7-NB3): -7.3620

Standard NCA - Chi-Square:
  Correlation (Pearson): r=-0.9918, p=0.0009
  Correlation (Spearman): r=-1.0000, p=0.0000
  Linear trend: slope=-0.190200, p=0.0009
  Best NB: 7, Worst NB: 3
  Improvement NB3→NB7: 79.67%
  Fold Change (NB7/NB3): 0.203x
  Absolute Difference (NB7-NB3): -0.7840

Standard NCA - Categorical MMD:
  Correlation (Pearson): r=-0.6838, p=0.2030
  Correlation (Spearman): r=-0.5000, p=0.3910
  Linear trend: slope=-0.061300, p=0.2030
  Best NB: 6, Worst NB: 3
  Improvement NB3→NB7: 29.61%
  Fold Change (NB7/NB3): 0.704x
  Absolute Difference (NB7-NB3): -0.2840

Standard NCA - Tumor Size Diff:
  Correlation (Pearson): r=-0.7467, p=0.1471
  Correlation (S

,Model Type,Metric,Mean_NB3,Mean_NB7,Improvement_3_to_7,Fold_Change_NB7_vs_NB3,Absolute_Diff_NB7_vs_NB3,Pearson_r,Pearson_p,Spearman_r,Spearman_p,Slope,Slope_p,Best_NB,Worst_NB
0,Standard NCA,KL Divergence,7.580,0.218,97.124011,0.028760,-7.362,-0.821757,0.087880,-1.000000,1.404265e-24,-1.5670,0.087880,7,3
1,Standard NCA,Chi-Square,0.984,0.200,79.674797,0.203252,-0.784,-0.991849,0.000882,-1.000000,1.404265e-24,-0.1902,0.000882,7,3
2,Standard NCA,Categorical MMD,0.959,0.675,29.614181,0.703858,-0.284,-0.683821,0.202992,-0.500000,3.910022e-01,-0.0613,0.202992,6,3
3,Standard NCA,Tumor Size Diff,1.119,0.246,78.016086,0.219839,-0.873,-0.746687,0.147092,-0.700000,1.881204e-01,-0.1749,0.147092,5,3
4,Standard NCA,Border Size Diff,0.457,0.457,0.000000,1.000000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,3,3
5,Standard NCA,Spatial Variance Diff,1.046,1.046,0.000000,1.000000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,3,3
6,Mixture NCA,KL Divergence,0.054,1.136,-2003.703704,21.037037,1.082,0.829019,0.082659,0.500000,3.910022e-01,0.3428,0.082659,5,6
7,Mixture NCA,Chi-Square,0.053,0.926,-1647.169811,17.471698,0.873,0.840468,0.074633,0.600000,2.847570e-01,0.2063,0.074633,5,7
8,Mixture NCA,Categorical MMD,0.055,0.192,-249.090909,3.490909,0.137,0.613328,0.271267,0.600000,2.847570e-01,0.0371,0.271267,5,6
9,Mixture NCA,Tumor Size Diff,0.441,0.640,-45.124717,1.451247,0.199,0.281585,0.646271,0.200000,7.470601e-01,0.0282,0.646271,5,7


**Spearman is NaN only for the two constant-value cases:**

- **Standard NCA – Border Size Diff:** all values = `0.457`  
- **Standard NCA – Spatial Variance Diff:** all values = `1.046`

**Why this happens:**

- Correlation requires variation in both variables. If one variable is constant, correlation is undefined.  
- The code detects constant values and sets both Pearson and Spearman to `NaN`, which is the correct behavior.


### Detailed visualization with absolute values, fold change, and differences

In [95]:
available_cols = trend_df.columns.tolist()
print(f"Available columns: {available_cols}\n")

# Select only the columns that exist
base_cols = ['Model Type', 'Metric', 'Mean_NB3', 'Mean_NB7', 'Improvement_3_to_7', 'Best_NB']
optional_cols = ['Fold_Change_NB7_vs_NB3', 'Absolute_Diff_NB7_vs_NB3']

cols_to_use = [col for col in base_cols if col in available_cols]
for col in optional_cols:
    if col in available_cols:
        cols_to_use.append(col)

detailed = trend_df[cols_to_use].copy()
detailed = detailed.dropna(subset=['Mean_NB3', 'Mean_NB7'])

if 'Fold_Change_NB7_vs_NB3' not in detailed.columns:
    detailed['Fold_Change_NB7_vs_NB3'] = detailed['Mean_NB7'] / detailed['Mean_NB3']
if 'Absolute_Diff_NB7_vs_NB3' not in detailed.columns:
    detailed['Absolute_Diff_NB7_vs_NB3'] = detailed['Mean_NB7'] - detailed['Mean_NB3']

# Sort by percentage improvement if available, otherwise by Mean_NB3
if 'Improvement_3_to_7' in detailed.columns:
    detailed = detailed.sort_values('Improvement_3_to_7')
else:
    detailed = detailed.sort_values('Mean_NB3')

print(f"{'Model Type':<25} {'Metric':<25} {'NB=3':<10} {'NB=7':<10} "
      f"{'Fold':<8} {'Diff':<10} {'% Change':<12} {'Best NB':<8} {'Direction'}")
print("-" * 125)

for _, row in detailed.iterrows():
    model = row['Model Type']
    metric = row['Metric']
    nb3 = row['Mean_NB3']
    nb7 = row['Mean_NB7']
    fold = row.get('Fold_Change_NB7_vs_NB3', nb7 / nb3 if nb3 != 0 else np.nan) #nan=spearman
    diff = row.get('Absolute_Diff_NB7_vs_NB3', nb7 - nb3)
    pct = row.get('Improvement_3_to_7', (nb3 - nb7) / nb3 * 100 if nb3 != 0 else np.nan)
    best = int(row['Best_NB']) if 'Best_NB' in row and not pd.isna(row['Best_NB']) else 'N/A'
    
    if pd.isna(pct):
        direction = "N/A"
    elif pct > 0:
        direction = "Improvement"
    elif pct < 0:
        direction = "Degradation"
    else:
        direction = "No change"
    
    fold_str = f"{fold:.2f}x" if not pd.isna(fold) else "N/A"
    diff_str = f"{diff:.4f}" if not pd.isna(diff) else "N/A"
    pct_str = f"{pct:>11.2f}%" if not pd.isna(pct) else "N/A"
    
    print(f"{model:<25} {metric:<25} {nb3:<10.4f} {nb7:<10.4f} "
          f"{fold_str:<8} {diff_str:<10} {pct_str:<12} {best:<8} {direction}")

Available columns: ['Model Type', 'Metric', 'Mean_NB3', 'Mean_NB7', 'Improvement_3_to_7', 'Fold_Change_NB7_vs_NB3', 'Absolute_Diff_NB7_vs_NB3', 'Pearson_r', 'Pearson_p', 'Spearman_r', 'Spearman_p', 'Slope', 'Slope_p', 'Best_NB', 'Worst_NB']

Model Type                Metric                    NB=3       NB=7       Fold     Diff       % Change     Best NB  Direction
-----------------------------------------------------------------------------------------------------------------------------
Mixture NCA               KL Divergence             0.0540     1.1360     21.04x   1.0820        -2003.70% 5        Degradation
Mixture NCA               Chi-Square                0.0530     0.9260     17.47x   0.8730        -1647.17% 5        Degradation
Mixture NCA               Categorical MMD           0.0550     0.1920     3.49x    0.1370         -249.09% 5        Degradation
Stochastic Mixture NCA    Categorical MMD           0.0170     0.0560     3.29x    0.0390         -229.41% 3        Degrad

In [96]:
print("\n" + "=" * 80)
print("LEGEND:")
print("  Fold: How many times the value changed (NB7 / NB3). For 'lower is better' metrics:")
print("        - < 1.0  = improvement (NB7 is better)")
print("        - > 1.0  = degradation (NB7 is worse)")
print("  Diff: Absolute difference (NB7 - NB3)")
print("  % Change: Percentage change (can be misleading for very small initial values)")
print("  Best NB: Best neighborhood size for this metric")
print("  Direction: Improvement / Degradation / No change")
print("=" * 80)


LEGEND:
  Fold: How many times the value changed (NB7 / NB3). For 'lower is better' metrics:
        - < 1.0  = improvement (NB7 is better)
        - > 1.0  = degradation (NB7 is worse)
  Diff: Absolute difference (NB7 - NB3)
  % Change: Percentage change (can be misleading for very small initial values)
  Best NB: Best neighborhood size for this metric
  Direction: Improvement / Degradation / No change


### Explanation of Improvement/Degradation Values

**Warning:** Percentage values can be misleading when the initial value (NB=3) is very small!

**How they are calculated:**
- Formula: `(value_NB3 - value_NB7) / value_NB3 * 100`
- For metrics where *lower is better* (KL Divergence, Chi-Square, etc.):
  - **Positive value** = improvement (NB7 is better than NB3)
  - **Negative value** = degradation (NB7 is worse than NB3)

**Problem with very small values:**
When the initial value is very small (e.g., 0.054), tiny absolute differences become huge percentage differences:
- Example: Mixture NCA – KL Divergence  
  - NB=3: 0.054  
  - NB=7: 1.136  
  - Improvement: **–2003.70%** (meaning the value increased by ~21×)

**Correct interpretation:**
- Look at the **absolute values** (NB3 vs NB7)
- Look at the **fold change** (how many times it changed: NB7 / NB3)
- Look at the **absolute difference** (NB7 – NB3)
- Percentages are useful only when the initial values are comparable

## Computational Complexity Analysis

Measure the computational cost for each neighborhood size:


In [97]:
model_types = ["standard", "nca_with_noise", "mixture", "stochastic"]
all_complexity_results = []

for model_type in model_types:
    print(f"Analizzando: {model_type.upper()}")
    
    try:
        complexity_df = analyzer.computational_complexity_analysis(
            n_samples=5, 
            model_type=model_type
        )
        
        if not complexity_df.empty:
            all_complexity_results.append(complexity_df)
            print(f"\n Completato: {model_type}")
            print(f" Righe: {len(complexity_df)}")
        else:
            print(f"\n Nessun dato disponibile per: {model_type}")
            
    except Exception as e:
        print(f"\n Errore durante l'analisi di {model_type}: {e}")
        import traceback
        traceback.print_exc()

if all_complexity_results:
    combined_complexity_df = pd.concat(all_complexity_results, ignore_index=True)
    
    print(combined_complexity_df)
    
    
    for model_type in combined_complexity_df['Model Type'].unique():
        model_data = combined_complexity_df[combined_complexity_df['Model Type'] == model_type]
        print(f"\n{model_type.upper()}:")
        print(f"  Tempo medio totale: {model_data['Mean Time (s)'].mean():.4f} ± {model_data['Mean Time (s)'].std():.4f} s")
        print(f"  Tempo medio per step: {model_data['Time per Step (ms)'].mean():.2f} ± {model_data['Time per Step (ms)'].std():.2f} ms")
        print(f"  NB più veloce: {model_data.loc[model_data['Mean Time (s)'].idxmin(), 'Neighborhood Size']}")
        print(f"  NB più lento: {model_data.loc[model_data['Mean Time (s)'].idxmax(), 'Neighborhood Size']}")
    
    print("Confronto tra modelli per ogni neighborhood size")
    
    for nb_size in sorted(combined_complexity_df['Neighborhood Size'].unique()):
        nb_data = combined_complexity_df[combined_complexity_df['Neighborhood Size'] == nb_size]
        print(f"\nNB={nb_size}:")
        for _, row in nb_data.iterrows():
            print(f"  {row['Model Type']:20s}: {row['Mean Time (s)']:.4f} s ({row['Time per Step (ms)']:.2f} ms/step)")
    
    output_path = analyzer.base_dir / "computational_complexity_all_models.csv"
    combined_complexity_df.to_csv(output_path, index=False)
    print(f"\n Risultati salvati in: {output_path}")
    
else:
    print("\n Nessun risultato disponibile!")

Analizzando: STANDARD

Computational Complexity Analysis - STANDARD

Testing NB_3 (standard)...
  Mean time: 0.0223 ± 0.0016 s
  Time per step: 0.64 ms
  Theoretical complexity factor: 1.00x

Testing NB_4 (standard)...
  Mean time: 0.0302 ± 0.0167 s
  Time per step: 0.86 ms
  Theoretical complexity factor: 1.78x
  Actual speedup vs NB=3: 0.74x

Testing NB_5 (standard)...
  Mean time: 0.0220 ± 0.0011 s
  Time per step: 0.63 ms
  Theoretical complexity factor: 2.78x
  Actual speedup vs NB=3: 1.01x

Testing NB_6 (standard)...
  Mean time: 0.0254 ± 0.0024 s
  Time per step: 0.73 ms
  Theoretical complexity factor: 4.00x
  Actual speedup vs NB=3: 0.88x

Testing NB_7 (standard)...
  Mean time: 0.0333 ± 0.0058 s
  Time per step: 0.95 ms
  Theoretical complexity factor: 5.44x
  Actual speedup vs NB=3: 0.67x


 Completato: standard
 Righe: 5
Analizzando: NCA_WITH_NOISE

Computational Complexity Analysis - NCA_WITH_NOISE

Testing NB_3 (nca_with_noise)...
  Mean time: 0.0280 ± 0.0028 s
  Time per

 Display computational complexity

In [99]:
if 'combined_complexity_df' in locals() and not combined_complexity_df.empty:
    df_to_plot = combined_complexity_df
else:
    df_to_plot = complexity_df.copy()
    if 'Model Type' not in df_to_plot.columns:
        df_to_plot['Model Type'] = 'Standard NCA'

fig = go.Figure()

colors = px.colors.qualitative.Set2

model_types = df_to_plot['Model Type'].unique()
for idx, model_type in enumerate(model_types):
    model_data = df_to_plot[df_to_plot['Model Type'] == model_type].copy()
    model_data = model_data.sort_values('Neighborhood Size')
    color = colors[idx % len(colors)]
    
    fig.add_trace(go.Scatter(
        x=model_data['Neighborhood Size'],
        y=model_data['Mean Time (s)'],
        mode='lines+markers',
        name=f'{model_type} - Mean Time',
        error_y=dict(type='data', array=model_data['Std Time (s)'], visible=True),
        line=dict(width=3, color=color),
        marker=dict(size=10),
        legendgroup=model_type
    ))
    
    fig.add_trace(go.Scatter(
        x=model_data['Neighborhood Size'],
        y=model_data['Normalized Time'],
        mode='lines+markers',
        name=f'{model_type} - Normalized',
        line=dict(width=2, color=color, dash='dash'),
        marker=dict(size=8, symbol='diamond'),
        legendgroup=model_type,
        showlegend=False  
    ))

fig.update_layout(
    title='Computational Complexity vs Neighborhood Size (All Models)',
    xaxis_title='Neighborhood Size',
    yaxis_title='Time (s) / Normalized Time',
    width=1200,
    height=700,
    template='plotly_white',
    hovermode='x unified',
    legend=dict(
        title='Model Type',
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.01
    )
)

fig.show()

# Production Analysis: Computational Complexity Assessment

Goal: Evaluate whether increasing the neighborhood size introduces computational complexity issues for a production model.

## Key Questions
1. Do execution times increase significantly with larger neighborhood sizes?
2. Are the execution times acceptable for a production application?
3. Is there a trade-off between biological performance and computational complexity?
4. Can we already draw definitive conclusions?

## Theoretical Complexity Explanation: O(n²)

- The O(n²) complexity comes from the 2D convolution.
- For each pixel in the grid (H × W pixels), a convolution with a kernel of size nb_size × nb_size is performed.
- The number of operations per pixel is proportional to nb_size².
- Therefore, the total complexity is O(H × W × nb_size²).
- For a grid of fixed size, this simplifies to O(nb_size²).


In [100]:
if 'combined_complexity_df' in locals() and not combined_complexity_df.empty:
    print("="*80)
    print("ANALISI COMPLESSITÀ COMPUTAZIONALE PER LA PRODUZIONE")
    print("="*80)
    print()
    
    # 1. Analisi scaling reale vs teorico
    print("1. SCALING REALE VS TEORICO")
    print("-" * 80)
    
    for model_type in combined_complexity_df['Model Type'].unique():
        model_data = combined_complexity_df[combined_complexity_df['Model Type'] == model_type].copy()
        model_data = model_data.sort_values('Neighborhood Size')
        
        # Calcola rapporto tra tempo reale e teorico
        nb3_time = model_data[model_data['Neighborhood Size'] == 3]['Mean Time (s)'].values[0]
        nb3_ops = model_data[model_data['Neighborhood Size'] == 3]['Theoretical O(n²)'].values[0]
        
        print(f"\n{model_type.upper()}:")
        print(f"  Baseline (NB=3): {nb3_time:.4f}s, Ops teoriche: {nb3_ops}")
        
        for _, row in model_data.iterrows():
            nb_size = row['Neighborhood Size']
            real_time = row['Mean Time (s)']
            theoretical_ops = row['Theoretical O(n²)']
            
            # Rapporto teorico atteso
            expected_ratio = theoretical_ops / nb3_ops
            # Rapporto reale osservato
            actual_ratio = real_time / nb3_time
            
            print(
                f"  NB={nb_size}: Tempo={real_time:.4f}s, "
                f"Teorico={expected_ratio:.2f}x, Reale={actual_ratio:.2f}x, "
                f"Efficienza={expected_ratio/actual_ratio:.2f}x"
            )
    
    print("\n" + "="*80)
    print("2. VALUTAZIONE TEMPI PER PRODUZIONE")
    print("-" * 80)
    
    # Soglie tipiche per produzione (esempio)
    thresholds = {
        'real_time': 0.1,  # 100ms per 35 step
        'per_step': 5.0,   # 5ms per step
        'throughput': 10   # 10 simulazioni/secondo
    }
    
    print(f"\nSoglie di riferimento per produzione:")
    print(f"  - Tempo totale < {thresholds['real_time']*1000:.0f}ms per 35 step")
    print(f"  - Tempo per step < {thresholds['per_step']:.1f}ms")
    print(f"  - Throughput > {thresholds['throughput']} simulazioni/secondo")
    print()
    
    production_ready = {}
    
    for model_type in combined_complexity_df['Model Type'].unique():
        model_data = combined_complexity_df[combined_complexity_df['Model Type'] == model_type]
        
        # NIENTE INDENT STRANO QUI
        max_time = model_data['Mean Time (s)'].max()
        max_time_per_step = model_data['Time per Step (ms)'].max()
        min_throughput = 1.0 / max_time  # simulazioni/secondo
        
        # Valuta ogni criterio individualmente
        time_ok = max_time < thresholds['real_time']
        step_ok = max_time_per_step < thresholds['per_step']
        throughput_ok = min_throughput > thresholds['throughput']
        
        # Calcola margini per criteri non soddisfatti
        time_margin = (max_time - thresholds['real_time']) / thresholds['real_time'] * 100 if not time_ok else 0
        step_margin = (max_time_per_step - thresholds['per_step']) / thresholds['per_step'] * 100 if not step_ok else 0
        throughput_margin = (thresholds['throughput'] - min_throughput) / thresholds['throughput'] * 100 if not throughput_ok else 0
        
        # Conta criteri soddisfatti
        criteria_met = sum([time_ok, step_ok, throughput_ok])
        
        # Logica flessibile: PRONTO se:
        # - Tutti i criteri sono soddisfatti, OPPURE
        # - Almeno 2 su 3 criteri sono soddisfatti E il terzo è entro il 30% della soglia
        max_margin = max(time_margin, step_margin, throughput_margin)
        is_ready = (criteria_met == 3) or (criteria_met >= 2 and max_margin <= 30)
        
        production_ready[model_type] = is_ready
        
        # Determina status con note (senza iconcine)
        if is_ready:
            if criteria_met == 3:
                status = "PRONTO"
                note = " - Tutti i criteri soddisfatti"
            else:
                status = "PRONTO (con margine)"
                note = f" - {criteria_met}/3 criteri soddisfatti, margine max: {max_margin:.1f}%"
        else:
            status = "DA VALUTARE"
            note = f" - Solo {criteria_met}/3 criteri soddisfatti"
            if max_margin > 30:
                note += f", margine critico: {max_margin:.1f}%"
        
        print(f"{model_type.upper()}: {status}{note}")
        print()
        
        # Mostra dettaglio per ogni criterio (senza ✓/✗)
        print("  Criteri di valutazione:")
        
        print(
            f"    - Tempo totale: {max_time*1000:.1f}ms "
            f"(soglia: {thresholds['real_time']*1000:.0f}ms)", 
            end=""
        )
        if not time_ok:
            print(f" - {time_margin:.1f}% sopra la soglia")
        else:
            print(" - OK")
        
        print(
            f"    - Tempo per step: {max_time_per_step:.2f}ms "
            f"(soglia: {thresholds['per_step']:.1f}ms)", 
            end=""
        )
        if not step_ok:
            print(f" - {step_margin:.1f}% sopra la soglia")
        else:
            print(" - OK")
        
        print(
            f"    - Throughput: {min_throughput:.1f} sim/s "
            f"(soglia: {thresholds['throughput']:.0f} sim/s)", 
            end=""
        )
        if not throughput_ok:
            print(f" - {throughput_margin:.1f}% sotto la soglia")
        else:
            print(" - OK")
        
        print()
    
    print("="*80)
    print("3. VARIAZIONE TEMPI CON NEIGHBORHOOD SIZE")
    print("-" * 80)
    
    for model_type in combined_complexity_df['Model Type'].unique():
        model_data = combined_complexity_df[combined_complexity_df['Model Type'] == model_type].copy()
        model_data = model_data.sort_values('Neighborhood Size')
        
        times = model_data['Mean Time (s)'].values
        nb_sizes = model_data['Neighborhood Size'].values
        
        # Calcola coefficiente di variazione
        cv = np.std(times) / np.mean(times) * 100
        
        # Calcola range (max - min) / min
        time_range_pct = (np.max(times) - np.min(times)) / np.min(times) * 100
        
        print(f"\n{model_type.upper()}:")
        print(f"  Coefficiente di variazione: {cv:.1f}%")
        print(f"  Range relativo: {time_range_pct:.1f}%")
        
        if time_range_pct < 20:
            print("  Variazione MINIMA: i tempi sono praticamente costanti")
        elif time_range_pct < 50:
            print("  Variazione MODERATA: aumento accettabile")
        else:
            print("  Variazione SIGNIFICATIVA: potrebbe essere un problema")
    
    all_ready = all(production_ready.values())
    some_variation = any(
        (
            combined_complexity_df[combined_complexity_df['Model Type'] == mt]['Mean Time (s)'].max() /
            combined_complexity_df[combined_complexity_df['Model Type'] == mt]['Mean Time (s)'].min() - 1
        ) > 0.3
        for mt in combined_complexity_df['Model Type'].unique()
    )

else:
    print("Errore: Dati di complessità computazionale non disponibili.")
    print("Eseguire prima la cella di analisi della complessità computazionale.")

ANALISI COMPLESSITÀ COMPUTAZIONALE PER LA PRODUZIONE

1. SCALING REALE VS TEORICO
--------------------------------------------------------------------------------

STANDARD:
  Baseline (NB=3): 0.0223s, Ops teoriche: 9
  NB=3: Tempo=0.0223s, Teorico=1.00x, Reale=1.00x, Efficienza=1.00x
  NB=4: Tempo=0.0302s, Teorico=1.78x, Reale=1.36x, Efficienza=1.31x
  NB=5: Tempo=0.0220s, Teorico=2.78x, Reale=0.99x, Efficienza=2.81x
  NB=6: Tempo=0.0254s, Teorico=4.00x, Reale=1.14x, Efficienza=3.51x
  NB=7: Tempo=0.0333s, Teorico=5.44x, Reale=1.49x, Efficienza=3.65x

NCA_WITH_NOISE:
  Baseline (NB=3): 0.0280s, Ops teoriche: 9
  NB=3: Tempo=0.0280s, Teorico=1.00x, Reale=1.00x, Efficienza=1.00x
  NB=4: Tempo=0.0267s, Teorico=1.78x, Reale=0.95x, Efficienza=1.86x
  NB=5: Tempo=0.0251s, Teorico=2.78x, Reale=0.90x, Efficienza=3.09x
  NB=6: Tempo=0.0266s, Teorico=4.00x, Reale=0.95x, Efficienza=4.20x
  NB=7: Tempo=0.0296s, Teorico=5.44x, Reale=1.06x, Efficienza=5.16x

MIXTURE:
  Baseline (NB=3): 0.0708s, Ops

## Interactive Visualizations

Create interactive visualizations with Plotly:


In [101]:
analyzer.create_visualizations()

print("\n Visualizations created")


Creating Visualizations

Saved: /Users/luigidaddario/Documents/GitHub/MNCA/experiments/results_extended/tissue_simulation_extended/analysis_plots/kl_divergence_boxplot.html
Saved: /Users/luigidaddario/Documents/GitHub/MNCA/experiments/results_extended/tissue_simulation_extended/analysis_plots/chi-square_boxplot.html
Saved: /Users/luigidaddario/Documents/GitHub/MNCA/experiments/results_extended/tissue_simulation_extended/analysis_plots/categorical_mmd_boxplot.html
Saved: /Users/luigidaddario/Documents/GitHub/MNCA/experiments/results_extended/tissue_simulation_extended/analysis_plots/tumor_size_diff_boxplot.html
Saved: /Users/luigidaddario/Documents/GitHub/MNCA/experiments/results_extended/tissue_simulation_extended/analysis_plots/border_size_diff_boxplot.html
Saved: /Users/luigidaddario/Documents/GitHub/MNCA/experiments/results_extended/tissue_simulation_extended/analysis_plots/spatial_variance_diff_boxplot.html
Saved: /Users/luigidaddario/Documents/GitHub/MNCA/experiments/results_exte

## Custom Visualizations in the Notebook

Create interactive visualizations directly in the notebook:


In [102]:
metric_cols = ['KL Divergence', 'Chi-Square', 'Categorical MMD', 
              'Tumor Size Diff', 'Border Size Diff', 'Spatial Variance Diff']

fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=metric_cols,
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

colors = px.colors.qualitative.Set2
df_parsed = analyzer.parse_metrics()

for idx, metric in enumerate(metric_cols):
    if metric not in df_parsed.columns:
        continue
    
    row = (idx // 3) + 1
    col = (idx % 3) + 1
    
    for model_idx, model_type in enumerate(df_parsed['Model Type'].unique()):
        model_data = df_parsed[df_parsed['Model Type'] == model_type]
        grouped = model_data.groupby('Neighborhood Size')[metric].agg(['mean', 'std'])
        
        sizes = grouped.index.values
        means = grouped['mean'].values
        stds = grouped['std'].values
        
        color = colors[model_idx % len(colors)]
        
        fig.add_trace(
            go.Scatter(
                x=sizes,
                y=means,
                mode='lines+markers',
                name=model_type if idx == 0 else '',
                line=dict(color=color, width=2),
                marker=dict(size=8, color=color),
                error_y=dict(type='data', array=stds, visible=True),
                showlegend=(idx == 0),
                hovertemplate=f'<b>{model_type}</b><br>' +
                            'Neighborhood Size: %{x}<br>' +
                            f'{metric}: %{{y:.4f}}<br>' +
                            '<extra></extra>'
            ),
            row=row, col=col
        )

fig.update_layout(
    title_text="Complete Dashboard: Performance by Neighborhood Size",
    height=1000,
    width=1800,
    font=dict(size=10),
    title_font_size=18,
    template='plotly_white'
)

fig.show()

## Finding best configuration

In [105]:
print("="*60)
print("Best configurations by metric")
print("="*60)

for metric in metric_cols:
    if metric not in df_parsed.columns:
        continue
    
    best_idx = df_parsed[metric].idxmin()
    best_row = df_parsed.loc[best_idx]
    print(f"\n{metric}:")
    print(f"  Best: {best_row['Model Type']} with NB={best_row['Neighborhood Size']}")
    print(f"  Value: {best_row[metric]:.4f}")

Best configurations by metric

KL Divergence:
  Best: Mixture NCA with NB=5
  Value: 0.0100

Chi-Square:
  Best: Mixture NCA with NB=5
  Value: 0.0100

Categorical MMD:
  Best: Stochastic Mixture NCA with NB=3
  Value: 0.0170

Tumor Size Diff:
  Best: Stochastic Mixture NCA with NB=4
  Value: 0.0760

Border Size Diff:
  Best: Stochastic Mixture NCA with NB=3
  Value: 0.0610

Spatial Variance Diff:
  Best: Stochastic Mixture NCA with NB=3
  Value: 0.1270


## Rule Specialization Analysis

Analyze how Mixture NCA and Stochastic Mixture NCA rules specialize (or lose specialization) with different neighborhood sizes.

**Hypothesis**: With larger neighborhood sizes, the perceived distributions become more complex, and rules may lose their ability to specialize on distinct distributions, leading to performance degradation.

**Analysis**:
1. Load Mixture NCA and Stochastic Mixture NCA models for each neighborhood size
2. Generate example states from histories
3. Analyze rule assignment probabilities for both model types
4. Calculate specialization metrics (entropy, diversity) for both
5. Visualize rule assignments across neighborhood sizes and compare both models


In [106]:
from mix_NCA.ExtendedMixtureNCA import ExtendedMixtureNCA
from mix_NCA.ExtendedMixtureNCANoise import ExtendedMixtureNCANoise
from mix_NCA.utils_simulations import grid_to_channels_batch, classification_update_net
from mix_NCA.TissueModel import ComplexCellType
import torch
from scipy.stats import entropy

In [107]:
# Load Mixture NCA and Stochastic Mixture NCA models for each neighborhood size
mix_nca_models = {}
stochastic_mix_nca_models = {}

def make_update_net_fn(device):
    def update_net_wrapper(n_channels, hidden_dims=128, n_channels_out=None, device_arg=None):
        return classification_update_net(n_channels, hidden_dims, n_channels_out, device=device)
    return update_net_wrapper

update_net_fn = make_update_net_fn(analyzer.device)

print("Loading Mixture NCA models for rule analysis...")
for nb_size in NEIGHBORHOOD_SIZES:
    exp_dir = Path(RESULTS_DIR) / "tissue_simulation_extended" / f"NB_{nb_size}"
    
    # Mixture NCA
    mix_path = exp_dir / 'mixture_nca.pt'
    if mix_path.exists():
        mix_nca = ExtendedMixtureNCA(
            update_nets=update_net_fn,
            hidden_dim=128,
            maintain_seed=False,
            use_alive_mask=False,
            state_dim=6,
            num_rules=5,
            residual=False,
            temperature=3,
            neighborhood_size=nb_size,
            device=analyzer.device
        )
        mix_nca.load_state_dict(torch.load(mix_path, map_location=analyzer.device, weights_only=True))
        mix_nca.eval()
        mix_nca_models[nb_size] = mix_nca
        print(f"Loaded Mixture NCA for NB={nb_size}")
    else:
        print(f"Mixture NCA not found for NB={nb_size}")
    
    # Stochastic Mixture NCA
    stoch_path = exp_dir / 'stochastic_mix_nca.pt'
    if stoch_path.exists():
        stochastic_mix_nca = ExtendedMixtureNCANoise(
            update_nets=update_net_fn,
            hidden_dim=128,
            maintain_seed=False,
            use_alive_mask=False,
            state_dim=6,
            num_rules=5,
            residual=False,
            temperature=3,
            neighborhood_size=nb_size,
            device=analyzer.device
        )
        stochastic_mix_nca.load_state_dict(torch.load(stoch_path, map_location=analyzer.device, weights_only=True))
        stochastic_mix_nca.eval()
        stochastic_mix_nca_models[nb_size] = stochastic_mix_nca
        print(f"Loaded Stochastic Mixture NCA for NB={nb_size}")
    else:
        print(f"Stochastic Mixture NCA not found for NB={nb_size}")

print(f"\nLoaded {len(mix_nca_models)} Mixture NCA models")
print(f"Loaded {len(stochastic_mix_nca_models)} Stochastic Mixture NCA models")

Loading Mixture NCA models for rule analysis...
Loaded Mixture NCA for NB=3
Loaded Stochastic Mixture NCA for NB=3
Loaded Mixture NCA for NB=4
Loaded Stochastic Mixture NCA for NB=4
Loaded Mixture NCA for NB=5
Loaded Stochastic Mixture NCA for NB=5
Loaded Mixture NCA for NB=6
Loaded Stochastic Mixture NCA for NB=6
Loaded Mixture NCA for NB=7
Loaded Stochastic Mixture NCA for NB=7

Loaded 5 Mixture NCA models
Loaded 5 Stochastic Mixture NCA models


Prepare example states from histories and use a few different states to get a representative sample

In [108]:
n_samples = 5
sample_indices = np.linspace(0, len(analyzer.histories) - 1, n_samples, dtype=int)

# Convert grid states to channel format
example_states = []
for idx in sample_indices:
    grid_state = analyzer.histories[idx][0]  # Initial state
    encoded_state = grid_to_channels_batch([grid_state], len(ComplexCellType), analyzer.device)
    example_states.append(encoded_state)

print(f"Prepared {len(example_states)} example states")
print(f"State shape: {example_states[0].shape}")

Prepared 5 example states
State shape: torch.Size([1, 6, 30, 30])


Analyze rule assignments for each neighborhood size

In [109]:
def analyze_rules(model, model_name, nb_size):
    """Analyze rule assignments for a given model"""
    all_entropies = []
    rule_usage_counts = {i: 0 for i in range(5)}
    
    with torch.no_grad():
        for state in example_states:
            # Get rule probabilities
            probs = model.get_rule_probabilities(state)  # [batch, num_rules, height, width]
            
            # Calculate entropy for each spatial position (higher entropy = less specialization)
            # probs shape: [1, 5, H, W]
            probs_flat = probs[0].permute(1, 2, 0).cpu().numpy()  # [H, W, 5]
            entropies = [entropy(p, base=2) for p in probs_flat.reshape(-1, 5)]
            all_entropies.extend(entropies)
            
            # Count which rule is most likely at each position
            max_rule = probs[0].argmax(dim=0).cpu().numpy()  # [H, W]
            for rule_idx in range(5):
                rule_usage_counts[rule_idx] += np.sum(max_rule == rule_idx)
    
    mean_entropy = np.mean(all_entropies)
    std_entropy = np.std(all_entropies)
    
    # Rule diversity: how evenly rules are used
    total_usage = sum(rule_usage_counts.values())
    rule_proportions = [count / total_usage for count in rule_usage_counts.values()]
    rule_diversity = entropy(rule_proportions, base=2)  # Max is log2(5) ≈ 2.32
    
    max_rule_usage = max(rule_proportions)
    min_rule_usage = min(rule_proportions)
    
    return {
        'Mean Entropy': mean_entropy,
        'Std Entropy': std_entropy,
        'Rule Diversity': rule_diversity,
        'Max Rule Usage': max_rule_usage,
        'Min Rule Usage': min_rule_usage,
        'Rule Proportions': rule_proportions
    }

rule_analysis = {
    'Model Type': [],
    'Neighborhood Size': [],
    'Mean Entropy': [],
    'Std Entropy': [],
    'Rule Diversity': [],
    'Max Rule Usage': [],
    'Min Rule Usage': []
}

print("Analyzing Mixture NCA models...")
for nb_size in NEIGHBORHOOD_SIZES:
    if nb_size not in mix_nca_models:
        continue
    
    metrics = analyze_rules(mix_nca_models[nb_size], 'Mixture NCA', nb_size)
    rule_analysis['Model Type'].append('Mixture NCA')
    rule_analysis['Neighborhood Size'].append(nb_size)
    rule_analysis['Mean Entropy'].append(metrics['Mean Entropy'])
    rule_analysis['Std Entropy'].append(metrics['Std Entropy'])
    rule_analysis['Rule Diversity'].append(metrics['Rule Diversity'])
    rule_analysis['Max Rule Usage'].append(metrics['Max Rule Usage'])
    rule_analysis['Min Rule Usage'].append(metrics['Min Rule Usage'])
    
    print(f"\nMixture NCA - NB={nb_size}:")
    print(f"  Mean Entropy: {metrics['Mean Entropy']:.4f} ± {metrics['Std Entropy']:.4f}")
    print(f"  Rule Diversity: {metrics['Rule Diversity']:.4f} (max={np.log2(5):.4f})")
    print(f"  Rule Usage: {[f'{p:.2%}' for p in metrics['Rule Proportions']]}")


for nb_size in NEIGHBORHOOD_SIZES:
    if nb_size not in stochastic_mix_nca_models:
        continue
    
    metrics = analyze_rules(stochastic_mix_nca_models[nb_size], 'Stochastic Mixture NCA', nb_size)
    rule_analysis['Model Type'].append('Stochastic Mixture NCA')
    rule_analysis['Neighborhood Size'].append(nb_size)
    rule_analysis['Mean Entropy'].append(metrics['Mean Entropy'])
    rule_analysis['Std Entropy'].append(metrics['Std Entropy'])
    rule_analysis['Rule Diversity'].append(metrics['Rule Diversity'])
    rule_analysis['Max Rule Usage'].append(metrics['Max Rule Usage'])
    rule_analysis['Min Rule Usage'].append(metrics['Min Rule Usage'])
    
    print(f"\nStochastic Mixture NCA - NB={nb_size}:")
    print(f"  Mean Entropy: {metrics['Mean Entropy']:.4f} ± {metrics['Std Entropy']:.4f}")
    print(f"  Rule Diversity: {metrics['Rule Diversity']:.4f} (max={np.log2(5):.4f})")
    print(f"  Rule Usage: {[f'{p:.2%}' for p in metrics['Rule Proportions']]}")

rule_df = pd.DataFrame(rule_analysis)
print("\n Rule Specialization Summary")
print(rule_df.to_string(index=False))


Analyzing Mixture NCA models...

Mixture NCA - NB=3:
  Mean Entropy: 0.0070 ± 0.0006
  Rule Diversity: 0.1301 (max=2.3219)
  Rule Usage: ['0.00%', '0.00%', '0.00%', '1.80%', '98.20%']

Mixture NCA - NB=4:
  Mean Entropy: 0.0044 ± 0.0077
  Rule Diversity: 0.1373 (max=2.3219)
  Rule Usage: ['0.00%', '0.00%', '98.07%', '1.93%', '0.00%']

Mixture NCA - NB=5:
  Mean Entropy: 0.0061 ± 0.0059
  Rule Diversity: 0.1451 (max=2.3219)
  Rule Usage: ['0.00%', '0.00%', '2.07%', '97.93%', '0.00%']

Mixture NCA - NB=6:
  Mean Entropy: 0.0063 ± 0.0056
  Rule Diversity: 0.1537 (max=2.3219)
  Rule Usage: ['97.78%', '2.22%', '0.00%', '0.00%', '0.00%']

Mixture NCA - NB=7:
  Mean Entropy: 0.0201 ± 0.0046
  Rule Diversity: 0.1632 (max=2.3219)
  Rule Usage: ['0.00%', '0.00%', '97.60%', '0.00%', '2.40%']

Stochastic Mixture NCA - NB=3:
  Mean Entropy: 0.0095 ± 0.0048
  Rule Diversity: 0.1301 (max=2.3219)
  Rule Usage: ['98.20%', '0.00%', '1.80%', '0.00%', '0.00%']

Stochastic Mixture NCA - NB=4:
  Mean Entrop

Visualize rule specialization metrics

In [110]:
# Visualize rule specialization metrics for both model types
colors = {'Mixture NCA': 'blue', 'Stochastic Mixture NCA': 'red'}

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Mean Entropy (Lower = More Specialized)', 
                    'Rule Diversity (Higher = More Diverse Usage)',
                    'Max Rule Usage (Lower = More Balanced)',
                    'Entropy vs Performance'),
    vertical_spacing=0.12,
    horizontal_spacing=0.15
)

# Get performance data for comparison
perf_data_mix = df_parsed[df_parsed['Model Type'] == 'Mixture NCA'].copy()
perf_data_stoch = df_parsed[df_parsed['Model Type'] == 'Stochastic Mixture NCA'].copy()

# Plot for each model type
for model_type in ['Mixture NCA', 'Stochastic Mixture NCA']:
    model_data = rule_df[rule_df['Model Type'] == model_type].copy()
    model_data = model_data.sort_values('Neighborhood Size')
    color = colors[model_type]
    
    # Plot 1: Mean Entropy
    fig.add_trace(
        go.Scatter(
            x=model_data['Neighborhood Size'],
            y=model_data['Mean Entropy'],
            mode='lines+markers',
            name=f'{model_type} - Entropy',
            error_y=dict(type='data', array=model_data['Std Entropy'], visible=True),
            line=dict(width=3, color=color),
            marker=dict(size=12),
            legendgroup=model_type
        ),
        row=1, col=1
    )
    
    # Plot 2: Rule Diversity
    fig.add_trace(
        go.Scatter(
            x=model_data['Neighborhood Size'],
            y=model_data['Rule Diversity'],
            mode='lines+markers',
            name=f'{model_type} - Diversity',
            line=dict(width=3, color=color, dash='dash' if model_type == 'Stochastic Mixture NCA' else 'solid'),
            marker=dict(size=12),
            legendgroup=model_type
        ),
        row=1, col=2
    )
    
    # Plot 3: Max Rule Usage
    fig.add_trace(
        go.Scatter(
            x=model_data['Neighborhood Size'],
            y=model_data['Max Rule Usage'],
            mode='lines+markers',
            name=f'{model_type} - Max Usage',
            line=dict(width=3, color=color, dash='dot' if model_type == 'Stochastic Mixture NCA' else 'solid'),
            marker=dict(size=12),
            legendgroup=model_type
        ),
        row=2, col=1
    )
    
    # Plot 4: Entropy vs KL Divergence (performance)
    perf_data = perf_data_mix if model_type == 'Mixture NCA' else perf_data_stoch
    if len(perf_data) > 0 and len(model_data) > 0:
        # Merge data
        merged = model_data.merge(perf_data[['Neighborhood Size', 'KL Divergence']], 
                                  on='Neighborhood Size', how='inner')
        if len(merged) > 0:
            fig.add_trace(
                go.Scatter(
                    x=merged['Mean Entropy'],
                    y=merged['KL Divergence'],
                    mode='markers+text',
                    text=[f'NB={nb}' for nb in merged['Neighborhood Size']],
                    textposition="top center",
                    name=f'{model_type} - Performance',
                    marker=dict(size=15, color=color),
                    legendgroup=model_type,
                    hovertemplate=f'<b>{model_type} - NB=%{{text}}</b><br>' +
                                  'Mean Entropy: %{x:.4f}<br>' +
                                  'KL Divergence: %{y:.4f}<br>' +
                                  '<extra></extra>'
                ),
                row=2, col=2
            )

# Add max diversity line
max_diversity = np.log2(5)
fig.add_hline(y=max_diversity, line_dash="dash", line_color="gray",
              annotation_text=f"Max diversity ({max_diversity:.2f})", row=1, col=2)

# Update axes
fig.update_xaxes(title_text="Neighborhood Size", row=1, col=1)
fig.update_xaxes(title_text="Neighborhood Size", row=1, col=2)
fig.update_xaxes(title_text="Neighborhood Size", row=2, col=1)
fig.update_xaxes(title_text="Mean Entropy", row=2, col=2)

fig.update_yaxes(title_text="Entropy (bits)", row=1, col=1)
fig.update_yaxes(title_text="Diversity (bits)", row=1, col=2)
fig.update_yaxes(title_text="Max Usage Proportion", row=2, col=1)
fig.update_yaxes(title_text="KL Divergence", row=2, col=2)

fig.update_layout(
    title_text="Rule Specialization Analysis: Mixture NCA vs Stochastic Mixture NCA",
    height=1000,
    width=1400,
    template='plotly_white',
    showlegend=True
)

fig.show()


Visualize actual rule assignments for a sample state and pick one example state to visualize

In [111]:
sample_state = example_states[0]

for model_type_name, models_dict, title_suffix in [
    ('Mixture NCA', mix_nca_models, 'Mixture NCA'),
    ('Stochastic Mixture NCA', stochastic_mix_nca_models, 'Stochastic Mixture NCA')
]:
    if len(models_dict) == 0:
        continue
    
    fig = make_subplots(
        rows=len(NEIGHBORHOOD_SIZES), 
        cols=6,  # Original state + 5 rules
        subplot_titles=[f'NB={nb_size}' if i == 0 else f'Rule {i-1}' 
                        for nb_size in NEIGHBORHOOD_SIZES for i in range(6)],
        vertical_spacing=0.05,
        horizontal_spacing=0.05
    )
    
    for row_idx, nb_size in enumerate(NEIGHBORHOOD_SIZES):
        if nb_size not in models_dict:
            continue
        
        model = models_dict[nb_size]
        
        with torch.no_grad():
            probs = model.get_rule_probabilities(sample_state)  # [1, 5, H, W]
        
        # Plot original state (first column)
        state_img = sample_state[0, :6].cpu().numpy().transpose(1, 2, 0)
        # Convert to RGB for visualization (take first 3 channels or create composite)
        state_vis = np.clip(state_img[:, :, :3], 0, 1) if state_img.shape[2] >= 3 else state_img[:, :, 0]
        
        fig.add_trace(
            go.Heatmap(
                z=state_vis[:, :, 0] if len(state_vis.shape) == 3 else state_vis,
                colorscale='gray',
                showscale=False
            ),
            row=row_idx+1, col=1
        )
        
        #Rule's probability map
        for rule_idx in range(5):
            rule_probs = probs[0, rule_idx].cpu().numpy()
            fig.add_trace(
                go.Heatmap(
                    z=rule_probs,
                    colorscale='Viridis',
                    showscale=(row_idx == 0 and rule_idx == 4),
                    colorbar=dict(len=0.2, y=0.5) if (row_idx == 0 and rule_idx == 4) else None
                ),
                row=row_idx+1, col=rule_idx+2
            )
    
    fig.update_layout(
        title_text=f"Rule Assignment Probabilities: {title_suffix}",
        height=200 * len(NEIGHBORHOOD_SIZES),
        width=1800,
        template='plotly_white'
    )
    
    fig.show()

In [112]:
print("\n Visualization Interpretation:")
print("More uniform colors = less specialization (rules are used similarly everywhere)")
print("More varied colors = more specialization (rules are used in specific regions)")
print("Compare how patterns change between Mixture NCA and Stochastic Mixture NCA")
print("Look for differences in rule usage patterns across neighborhood sizes")


 Visualization Interpretation:
More uniform colors = less specialization (rules are used similarly everywhere)
More varied colors = more specialization (rules are used in specific regions)
Compare how patterns change between Mixture NCA and Stochastic Mixture NCA
Look for differences in rule usage patterns across neighborhood sizes


## Statistical Significance Tests

Perform formal statistical tests to determine if differences between neighborhood sizes are statistically significant:

1. **Kruskal-Wallis Test**: Tests if there are any significant differences across all neighborhood sizes (non-parametric ANOVA)
2. **Post-hoc Pairwise Comparisons**: Mann-Whitney U tests to identify which specific neighborhood sizes differ significantly


In [125]:
statistical_results = analyzer.statistical_significance_tests(alpha=0.05, correction_method='bonferroni', include_effect_size=True)

# Display results as DataFrame
if not statistical_results.empty:
    print("\n" + "="*80)
    print("STATISTICAL TEST RESULTS SUMMARY")
    print("="*80)
    
    # Show Kruskal-Wallis results
    kw_results = statistical_results[statistical_results['Test'] == 'Kruskal-Wallis']
    print("\nKruskal-Wallis Test Results (Overall differences):")
    if 'Effect_Size' in kw_results.columns:
        print(kw_results[['Model Type', 'Metric', 'H_statistic', 'p_value', 'Effect_Size', 'Significant']].to_string(index=False))
    else:
        print(kw_results[['Model Type', 'Metric', 'H_statistic', 'p_value', 'Significant']].to_string(index=False))
    
    # Show significant post-hoc results
    posthoc_results = statistical_results[
        (statistical_results['Test'] != 'Kruskal-Wallis') & 
        (statistical_results['Significant'] == True)
    ]
    
    if len(posthoc_results) > 0:
        print("\n\nSignificant Pairwise Differences (Mann-Whitney U):")
        if 'Effect_Size' in posthoc_results.columns:
            print(posthoc_results[['Model Type', 'Metric', 'Test', 'p_value', 'Effect_Size', 'Significance']].to_string(index=False))
        else:
            print(posthoc_results[['Model Type', 'Metric', 'Test', 'p_value', 'Significance']].to_string(index=False))
    else:
        print("\n\nNo significant pairwise differences found after correction.")
else:
    print("Warning: No statistical test results available. Make sure raw data files exist.")



Statistical Significance Tests

Significance level (alpha): 0.05
Multiple comparison correction: bonferroni
Note: With 5 groups, there are 10 pairwise comparisons.
Bonferroni corrected alpha = 0.05/10 = 0.005000

    This may indicate deterministic behavior or insufficient variability.
  Note: Complete separation in Standard NCA - KL Divergence (NB3 > NB4)
  Note: Complete separation in Standard NCA - KL Divergence (NB3 > NB5)
  Note: Complete separation in Standard NCA - KL Divergence (NB3 > NB6)
  Note: Complete separation in Standard NCA - KL Divergence (NB3 > NB7)
  Note: Complete separation in Standard NCA - KL Divergence (NB4 > NB5)
  Note: Complete separation in Standard NCA - KL Divergence (NB4 > NB6)
  Note: Complete separation in Standard NCA - KL Divergence (NB4 > NB7)
  Note: Complete separation in Standard NCA - KL Divergence (NB5 > NB6)
  Note: Complete separation in Standard NCA - KL Divergence (NB5 > NB7)
  Note: Complete separation in Standard NCA - KL Divergence (NB6

## Decision Table: When to Use Which Neighborhood Size?

Based on all the analyses performed, this table provides recommendations for choosing the optimal neighborhood size based on your priorities:


In [124]:
# Create comprehensive decision table
def create_decision_table(analyzer, trend_df, complexity_df, statistical_results):
    """Create a decision table with recommendations"""
    
    # Get best configurations
    df_parsed = analyzer.parse_metrics()
    metric_cols = ['KL Divergence', 'Chi-Square', 'Categorical MMD', 
                  'Tumor Size Diff', 'Border Size Diff', 'Spatial Variance Diff']
    
    # Calculate composite scores (normalized performance / normalized time)
    decision_data = []
    
    for model_type in df_parsed['Model Type'].unique():
        model_data = df_parsed[df_parsed['Model Type'] == model_type].copy()
        model_complexity = complexity_df[complexity_df['Model Type'] == model_type.lower().replace(' ', '_')]
        
        if len(model_complexity) == 0:
            # Try alternative naming
            model_name_map = {
                'Standard NCA': 'standard',
                'Mixture NCA': 'mixture',
                'Stochastic Mixture NCA': 'stochastic',
                'NCA with Noise': 'nca_with_noise'
            }
            model_complexity = complexity_df[complexity_df['Model Type'] == model_name_map.get(model_type, '')]
        
        for nb_size in sorted(model_data['Neighborhood Size'].unique()):
            nb_data = model_data[model_data['Neighborhood Size'] == nb_size]
            nb_complexity = model_complexity[model_complexity['Neighborhood Size'] == nb_size]
            
            if len(nb_data) == 0 or len(nb_complexity) == 0:
                continue
            
            # Calculate normalized performance score (lower is better for all metrics)
            performance_scores = []
            for metric in metric_cols:
                if metric in nb_data.columns:
                    value = nb_data[metric].iloc[0]
                    # Normalize by min/max across all neighborhood sizes for this model
                    all_values = model_data[metric].values
                    if len(all_values) > 0 and np.max(all_values) > np.min(all_values):
                        normalized = (value - np.min(all_values)) / (np.max(all_values) - np.min(all_values))
                        performance_scores.append(normalized)
            
            avg_performance = np.mean(performance_scores) if performance_scores else np.nan
            
            # Get computational time
            mean_time = nb_complexity['Mean Time (s)'].iloc[0] if len(nb_complexity) > 0 else np.nan
            time_per_step = nb_complexity['Time per Step (ms)'].iloc[0] if len(nb_complexity) > 0 else np.nan
            
            # Calculate efficiency score (lower performance score / normalized time)
            if not np.isnan(mean_time) and not np.isnan(avg_performance) and mean_time > 0:
                baseline_time = model_complexity[model_complexity['Neighborhood Size'] == 3]['Mean Time (s)'].iloc[0] if len(model_complexity[model_complexity['Neighborhood Size'] == 3]) > 0 else mean_time
                time_ratio = mean_time / baseline_time if baseline_time > 0 else 1.0
                efficiency_score = avg_performance / time_ratio  # Lower is better
            else:
                efficiency_score = np.nan
            
            # Check if this is best for any metric
            best_for = []
            for metric in metric_cols:
                if metric in model_data.columns:
                    best_nb = model_data.loc[model_data[metric].idxmin(), 'Neighborhood Size']
                    if best_nb == nb_size:
                        best_for.append(metric)
            
            # Check statistical significance
            is_significant = False
            if not statistical_results.empty:
                kw_test = statistical_results[
                    (statistical_results['Model Type'] == model_type) &
                    (statistical_results['Test'] == 'Kruskal-Wallis')
                ]
                if len(kw_test) > 0:
                    is_significant = kw_test['Significant'].any()
            
            decision_data.append({
                'Model Type': model_type,
                'Neighborhood Size': nb_size,
                'Avg Performance Score': avg_performance,
                'Mean Time (s)': mean_time,
                'Time per Step (ms)': time_per_step,
                'Efficiency Score': efficiency_score,
                'Best For': ', '.join(best_for[:2]) if best_for else 'None',
                'Statistically Significant': is_significant
            })
    
    decision_df = pd.DataFrame(decision_data)

    
    for model_type in decision_df['Model Type'].unique():
        model_decisions = decision_df[decision_df['Model Type'] == model_type].copy()
        model_decisions = model_decisions.sort_values('Efficiency Score', na_position='last')
        
        print(f"\n{model_type.upper()}")
        print("-" * 100)
        print(f"{'NB':<5} {'Perf Score':<12} {'Time (s)':<12} {'Time/Step (ms)':<15} {'Efficiency':<12} {'Best For':<40} {'Significant':<12}")
        print("-" * 100)
        
        for _, row in model_decisions.iterrows():
            perf = f"{row['Avg Performance Score']:.4f}" if not pd.isna(row['Avg Performance Score']) else "N/A"
            time_s = f"{row['Mean Time (s)']:.4f}" if not pd.isna(row['Mean Time (s)']) else "N/A"
            time_ms = f"{row['Time per Step (ms)']:.2f}" if not pd.isna(row['Time per Step (ms)']) else "N/A"
            eff = f"{row['Efficiency Score']:.4f}" if not pd.isna(row['Efficiency Score']) else "N/A"
            best = row['Best For'][:38] + ".." if len(row['Best For']) > 40 else row['Best For']
            sig = "Yes" if row['Statistically Significant'] else "No"
            
            print(f"{int(row['Neighborhood Size']):<5} {perf:<12} {time_s:<12} {time_ms:<15} {eff:<12} {best:<40} {sig:<12}")
        
        # Recommendations
        best_efficiency = model_decisions.loc[model_decisions['Efficiency Score'].idxmin()] if not model_decisions['Efficiency Score'].isna().all() else None
        best_performance = model_decisions.loc[model_decisions['Avg Performance Score'].idxmin()] if not model_decisions['Avg Performance Score'].isna().all() else None
        fastest = model_decisions.loc[model_decisions['Mean Time (s)'].idxmin()] if not model_decisions['Mean Time (s)'].isna().all() else None
        
        print(f"\n  Recommendations for {model_type}:")
        if best_efficiency is not None:
            print(f"    • Best Efficiency (Performance/Time): NB={int(best_efficiency['Neighborhood Size'])}")
        if best_performance is not None:
            print(f"    • Best Performance: NB={int(best_performance['Neighborhood Size'])}")
        if fastest is not None:
            print(f"    • Fastest: NB={int(fastest['Neighborhood Size'])}")
    
    return decision_df

# Create and display decision table
decision_table = create_decision_table(
    analyzer, 
    trend_df, 
    combined_complexity_df if 'combined_complexity_df' in locals() else None, 
    statistical_results if 'statistical_results' in locals() else pd.DataFrame()
)


STANDARD NCA
----------------------------------------------------------------------------------------------------
NB    Perf Score   Time (s)     Time/Step (ms)  Efficiency   Best For                                 Significant 
----------------------------------------------------------------------------------------------------
7     0.0540       0.0333       0.95            0.0362       KL Divergence, Chi-Square                Yes         
6     0.1361       0.0254       0.73            0.1195       Categorical MMD                          Yes         
4     0.3079       0.0302       0.86            0.2270       None                                     Yes         
5     0.2724       0.0220       0.63            0.2759       Tumor Size Diff                          Yes         
3     1.0000       0.0223       0.64            1.0000       Border Size Diff, Spatial Variance Diff  Yes         

  Recommendations for Standard NCA:
    • Best Efficiency (Performance/Time): NB=7
    • Best